# YOLOv5 GradCAM++ Demo for Anterior Segment Disease Classification

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kitaguchi77/yolov5-gradcam-corneai/blob/main/notebooks/demo.ipynb)

This notebook demonstrates how to use the YOLOv5 GradCAM++ and Cut-and-Paste validation tools for anterior segment disease classification analysis.

**Note**: This notebook is designed to run on Google Colab. If running locally, adjust the setup steps accordingly.

## Important Notes for Google Colab Users

1. **Model Weights**: You need to provide YOLOv5 model weights trained on corneal images
   - Upload directly when prompted, or
   - Place in Google Drive at: `/content/drive/MyDrive/YOLOv5_weights/last.pt`

2. **Test Images**: For actual analysis, you need slit-lamp photographs
   - The demo creates placeholder data for illustration purposes

3. **CPU Recommended**: Based on the reference implementation, CPU gives more consistent results than GPU

4. **Dependencies**: All required packages are installed automatically in the setup cell

## 0. Google Colab Setup

In [ ]:
# Check if running on Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running on Google Colab")
    
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Change to content directory
    %cd /content
    
    # Install required packages directly (based on reference_code.txt)
    print("\nInstalling required packages...")
    !pip uninstall deep_utils -y
    !pip install -U git+https://github.com/pooya-mohammadi/deep_utils.git --q
    !pip install torch torchvision --q
    !pip install opencv-python --q
    !pip install matplotlib pandas numpy tqdm scipy statsmodels seaborn plotly --q
    !pip install PyYAML Pillow scikit-image --q
    
    # Clone the yolov5-gradcam repository
    print("\nCloning yolov5-gradcam repository...")
    !git clone https://github.com/pooya-mohammadi/yolov5-gradcam
    
    # Also clone our repository for the custom modules
    print("\nCloning project repository...")
    !git clone https://github.com/kitaguchi77/yolov5-gradcam-corneai.git
    
    # Set up paths
    import os
    os.chdir('/content/yolov5-gradcam-corneai')
    
    # Clone YOLOv5 (CorneAI fork)
    if not os.path.exists('yolov5'):
        !git clone https://github.com/modafone/corneaai.git yolov5
    
    print("\nSetup complete!")
    
    # Optional: Upload model weights to Colab
    from google.colab import files
    print("\nPlease upload your YOLOv5 model weights file (or skip to use sample data):")
    print("Note: You can also place the weights file in your Google Drive")
    
    use_upload = input("Upload weights file? (y/n): ")
    if use_upload.lower() == 'y':
        uploaded = files.upload()
        if uploaded:
            weights_filename = list(uploaded.keys())[0]
            !mkdir -p models
            !mv {weights_filename} models/
            print(f"Model weights saved to models/{weights_filename}")
    else:
        print("Skipping upload - you can specify a Google Drive path later")
        weights_filename = None
else:
    print("Running locally - skipping Colab setup")

### Using the reference implementation approach

Based on the reference code from https://github.com/pooya-mohammadi/yolov5-gradcam

### Alternative: Use sample data from GitHub

In [ ]:
# Alternative: Use Google Drive paths or download sample data
if IN_COLAB:
    # Option 1: Use model from Google Drive (if you have it there)
    gdrive_model_path = "/content/drive/MyDrive/YOLOv5_weights/last.pt"  # Update this path
    
    if os.path.exists(gdrive_model_path):
        print(f"Found model in Google Drive: {gdrive_model_path}")
        weights_filename = "last.pt"
        !mkdir -p models
        !cp {gdrive_model_path} models/
    elif weights_filename is None:
        # Option 2: Download a sample model (you would need to host this)
        print("No model found. Using CPU device for demo...")
        print("Note: For actual inference, you need to provide model weights")
        
    # Create sample data directory
    !mkdir -p data/sample_images
    
    # For demo purposes, we'll create synthetic sample data
    print("\nCreating sample data for demonstration...")
    
    # Note: In a real scenario, you would download actual cornea images
    # For now, we'll create placeholder entries
    import pandas as pd
    
    sample_data = []
    classes = ["Normal", "Infectious keratitis", "Non-infectious keratitis", "Scar", "Tumor", 
               "Deposit", "APAC", "Lens opacity", "Bullous keratopathy"]
    
    # Create dummy entries (in practice, these would be real image paths)
    for i in range(10):
        sample_data.append({
            'image_path': f'data/sample_images/sample_{i}.jpg',
            'label': i % 9,
            'class_name': classes[i % 9]
        })
    
    df_sample = pd.DataFrame(sample_data)
    df_sample.to_csv('data/sample_images.csv', index=False)
    print("Sample data CSV created")
    
    # Note about the reference implementation
    print("\n" + "="*50)
    print("NOTE: This notebook is based on the reference implementation")
    print("from: https://github.com/pooya-mohammadi/yolov5-gradcam")
    print("For production use, ensure you have:")
    print("1. Actual YOLOv5 model weights trained on cornea images")
    print("2. Real test images from slit-lamp photography")
    print("3. Proper annotations for cut-and-paste validation")
    print("="*50)

## 1. Setup and Imports

In [ ]:
import sys
from pathlib import Path

# For Colab, we need to add the correct paths
if IN_COLAB:
    # Add paths for the cloned repositories
    sys.path.append('/content/yolov5-gradcam-cornea/Code')
    sys.path.append('/content/yolov5-gradcam')
    
    # Also add the yolov5 directory
    sys.path.append('/content/yolov5-gradcam-cornea/Code/yolov5')

# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from IPython.display import Image, display
import warnings
warnings.filterwarnings('ignore')

# Import torch
import torch
import torchvision

# For Colab, we'll create simplified versions of the modules if they're not available
try:
    from models.yolov5_model import YOLOv5Model
    from models.yolov5_gradcam import YOLOv5GradCAMPlusPlus
    from validation.cut_and_paste import CutAndPasteValidator
    from utils.data_loader import DataLoader
    from utils.metrics import MetricsCalculator
    from analysis.visualization import Visualizer
except ImportError as e:
    print(f"Import error: {e}")
    print("Note: Some modules may not be available. Using simplified demo mode.")
    
    # Define placeholder classes for demo
    class YOLOv5Model:
        def __init__(self, weights_path, device='cpu'):
            self.weights_path = weights_path
            self.device = device
            self.class_names = {
                0: "Normal",
                1: "Infectious keratitis", 
                2: "Non-infectious keratitis",
                3: "Scar",
                4: "Tumor",
                5: "Deposit",
                6: "APAC",
                7: "Lens opacity",
                8: "Bullous keratopathy"
            }
            print(f"Demo mode: YOLOv5Model initialized with {weights_path}")

# Set matplotlib backend for Colab
if IN_COLAB:
    %matplotlib inline

print("Import complete!")

## 2. Load Model and Data

In [ ]:
# Configuration
if IN_COLAB:
    # Colab paths - check if we have a weights file
    if 'weights_filename' in locals() and weights_filename:
        WEIGHTS_PATH = f"models/{weights_filename}"
    else:
        # Try Google Drive path
        WEIGHTS_PATH = "/content/drive/MyDrive/YOLOv5_weights/last.pt"
        
    TEST_DATA_PATH = "data/sample_images.csv"
    SAMPLE_IMAGES_DIR = "data/sample_images"
    
    # Check if weights exist
    import os
    if not os.path.exists(WEIGHTS_PATH):
        print(f"Warning: Model weights not found at {WEIGHTS_PATH}")
        print("You can:")
        print("1. Upload your model weights using the file upload in the setup cell")
        print("2. Place your weights in Google Drive at: /content/drive/MyDrive/YOLOv5_weights/")
        print("3. Continue in demo mode without actual inference")
        WEIGHTS_PATH = None
else:
    # Local paths - update these
    WEIGHTS_PATH = "path/to/yolov5_weights.pt"
    TEST_DATA_PATH = "path/to/test_data.csv"

# Device selection - based on reference_code.txt, CPU is recommended for consistency
DEVICE = "cpu"  # CPU recommended for reproducible results
print(f"Using device: {DEVICE}")
print("Note: Based on the reference implementation, CPU is recommended for consistent results")

# Load model if weights are available
if WEIGHTS_PATH and os.path.exists(WEIGHTS_PATH):
    print(f"\nLoading YOLOv5 model from {WEIGHTS_PATH}...")
    try:
        model = YOLOv5Model(WEIGHTS_PATH, device=DEVICE)
        print("Model loaded successfully!")
    except Exception as e:
        print(f"Error loading model: {e}")
        model = None
else:
    print("\nNo model weights available - running in demo mode")
    model = None

# Load or create test data
print(f"\nLoading test data...")

try:
    data_loader = DataLoader()
    test_images = data_loader.load_image_list(TEST_DATA_PATH)
    print(f"Loaded {len(test_images)} test images")
except:
    print("Using demo data")
    # Create demo data structure
    from types import SimpleNamespace
    test_images = [
        SimpleNamespace(
            image_path=f"sample_{i}.jpg",
            label=i % 9,
            class_name=["Normal", "Infectious keratitis", "Non-infectious keratitis", 
                       "Scar", "Tumor", "Deposit", "APAC", "Lens opacity", 
                       "Bullous keratopathy"][i % 9]
        ) for i in range(5)
    ]

if model:
    print(f"\nClasses: {list(model.class_names.values())}")
else:
    print("\nDisease classes:")
    for i in range(9):
        print(f"{i}: {test_images[0].class_name if i == 0 else test_images[i % len(test_images)].class_name}")

## 3. Single Image Analysis

In [ ]:
# Select a sample image
if len(test_images) > 0:
    sample_image = test_images[0]
    print(f"Analyzing: {sample_image.image_path}")
    print(f"True class: {sample_image.class_name}")
    
    # Display the image if in Colab
    if IN_COLAB:
        from IPython.display import Image as IPImage
        display(IPImage(sample_image.image_path, width=400))
    
    # Run inference
    results = model.predict(sample_image.image_path)
    
    if len(results['classes']) > 0:
        pred_class_idx = results['classes'][results['scores'].argmax()]
        pred_class_name = model.class_names[pred_class_idx]
        confidence = results['scores'].max()
        
        print(f"\nPredicted class: {pred_class_name} (confidence: {confidence:.3f})")
    else:
        print("\nNo detection")
        pred_class_idx = 0  # Default to first class for demo
else:
    print("No test images loaded. Please check your data path.")

# Demo mode message if no model is loaded
if len(test_images) > 0 and model is None:
    print("="*60)
    print("DEMO MODE: No model loaded")
    print("This section shows what the analysis would look like with a real model")
    print("To run actual inference, please provide model weights")
    print("="*60)
    
    # Create dummy results for demonstration
    sample_image = test_images[0]
    print(f"\nDemo image: {sample_image.image_path}")
    print(f"True class: {sample_image.class_name}")
    
    # Simulate prediction
    pred_class_idx = 1  # Demo prediction
    pred_class_name = "Infectious keratitis"
    confidence = 0.89
    
    print(f"\nSimulated prediction: {pred_class_name} (confidence: {confidence:.3f})")
    
elif len(test_images) > 0 and model is not None:
    # Actual inference with real model
    sample_image = test_images[0]
    print(f"Analyzing: {sample_image.image_path}")
    print(f"True class: {sample_image.class_name}")
    
    # Display the image if it exists and in Colab
    if IN_COLAB and os.path.exists(sample_image.image_path):
        from IPython.display import Image as IPImage
        display(IPImage(sample_image.image_path, width=400))
    
    # Run inference
    results = model.predict(sample_image.image_path)
    
    if len(results['classes']) > 0:
        pred_class_idx = results['classes'][results['scores'].argmax()]
        pred_class_name = model.class_names[pred_class_idx]
        confidence = results['scores'].max()
        
        print(f"\nPredicted class: {pred_class_name} (confidence: {confidence:.3f})")
    else:
        print("\nNo detection")
        pred_class_idx = 0  # Default to first class for demo
else:
    print("No test images available")

In [ ]:
# Initialize GradCAM++
target_layers = ['17', '20', '23']
gradcam = YOLOv5GradCAMPlusPlus(model, target_layers)

# Generate CAMs for all target layers
print("Generating GradCAM++ visualizations...")
cams = gradcam.generate_all_cams(sample_image.image_path, pred_class_idx)

# Load original image
img = cv2.imread(sample_image.image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Visualize CAMs for each layer
fig, axes = plt.subplots(1, len(target_layers) + 1, figsize=(20, 5))

# Original image
axes[0].imshow(img)
axes[0].set_title('Original Image', fontsize=14)
axes[0].axis('off')

# CAM overlays
for idx, (layer, (cam, metadata)) in enumerate(cams.items()):
    overlay = gradcam.visualize_cam(img, cam, alpha=0.5)
    axes[idx + 1].imshow(overlay)
    axes[idx + 1].set_title(f'Layer {layer}\nAOI_50: {metadata["aoi_50"]:.3f}', fontsize=14)
    axes[idx + 1].axis('off')

plt.tight_layout()
plt.show()

# Print statistics
print("\nLayer-wise AOI_50 values:")
for layer, (_, metadata) in cams.items():
    print(f"  Layer {layer}: {metadata['aoi_50']:.3f}")

# GradCAM++ Visualization
if model is not None:
    # Real GradCAM++ with actual model
    target_layers = ['17', '20', '23']
    gradcam = YOLOv5GradCAMPlusPlus(model, target_layers)
    
    print("Generating GradCAM++ visualizations...")
    cams = gradcam.generate_all_cams(sample_image.image_path, pred_class_idx)
    
    # Load original image
    img = cv2.imread(sample_image.image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
else:
    # Demo mode - create synthetic visualizations
    print("DEMO MODE: Showing synthetic GradCAM++ visualizations")
    print("With a real model, these would show actual attention patterns")
    
    # Create a dummy image
    img = np.ones((480, 640, 3), dtype=np.uint8) * 200
    cv2.circle(img, (320, 240), 150, (100, 150, 200), -1)  # Simulate cornea
    
    # Create synthetic CAMs for demonstration
    target_layers = ['17', '20', '23']
    cams = {}
    
    for i, layer in enumerate(target_layers):
        # Create progressively more focused attention maps
        cam = np.zeros((480, 640), dtype=np.float32)
        center_x, center_y = 320, 240
        radius = 150 - i * 40  # Decreasing radius for deeper layers
        
        y, x = np.ogrid[:480, :640]
        mask = (x - center_x)**2 + (y - center_y)**2 <= radius**2
        cam[mask] = np.random.rand(*np.sum(mask)) * 0.5 + 0.5
        
        # Add some noise to make it realistic
        cam = cv2.GaussianBlur(cam, (21, 21), 0)
        
        # Simulate metadata
        metadata = {
            'aoi_50': 0.3 - i * 0.08,  # Decreasing AOI for deeper layers
            'layer': layer
        }
        
        cams[layer] = (cam, metadata)

# Visualize CAMs for each layer
fig, axes = plt.subplots(1, len(target_layers) + 1, figsize=(20, 5))

# Original image
axes[0].imshow(img)
axes[0].set_title('Original Image', fontsize=14)
axes[0].axis('off')

# CAM overlays
for idx, (layer, (cam, metadata)) in enumerate(cams.items()):
    # Create overlay
    cam_resized = cv2.resize(cam, (img.shape[1], img.shape[0]))
    cam_normalized = (cam_resized - cam_resized.min()) / (cam_resized.max() - cam_resized.min() + 1e-8)
    
    # Apply colormap
    cam_colored = plt.cm.jet(cam_normalized)[:, :, :3]
    overlay = img / 255.0 * 0.5 + cam_colored * 0.5
    
    axes[idx + 1].imshow(overlay)
    axes[idx + 1].set_title(f'Layer {layer}\nAOI_50: {metadata["aoi_50"]:.3f}', fontsize=14)
    axes[idx + 1].axis('off')

plt.tight_layout()
plt.show()

# Print statistics
print("\nLayer-wise AOI_50 values:")
for layer, (_, metadata) in cams.items():
    print(f"  Layer {layer}: {metadata['aoi_50']:.3f}")

## 5. Batch Analysis - AOI Distribution

In [ ]:
# Analyze multiple images
n_samples = min(20, len(test_images))  # Analyze first 20 images
aoi_results = []

print(f"Analyzing {n_samples} images...")
for img_data in test_images[:n_samples]:
    # Run inference
    results = model.predict(img_data.image_path)
    
    if len(results['classes']) > 0:
        pred_class = results['classes'][results['scores'].argmax()]
        
        # Generate CAM for layer 23 (most informative according to paper)
        cam, metadata = gradcam.generate_cam(img_data.image_path, pred_class, '23')
        
        aoi_results.append({
            'image': Path(img_data.image_path).name,
            'true_class': img_data.class_name,
            'pred_class': model.class_names[pred_class],
            'correct': pred_class == img_data.label,
            'aoi_50': metadata['aoi_50']
        })

# Convert to DataFrame
df_aoi = pd.DataFrame(aoi_results)

# Display results
print("\nSample results:")
print(df_aoi.head(10))

# Plot AOI distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# AOI by class
df_aoi.boxplot(column='aoi_50', by='true_class', ax=ax1, rot=45)
ax1.set_title('AOI_50 Distribution by True Class')
ax1.set_xlabel('Disease Class')
ax1.set_ylabel('AOI_50')

# AOI by correctness
df_aoi.boxplot(column='aoi_50', by='correct', ax=ax2)
ax2.set_title('AOI_50 by Prediction Correctness')
ax2.set_xlabel('Correct Prediction')
ax2.set_ylabel('AOI_50')

plt.tight_layout()
plt.show()

# Summary statistics
print("\nSummary statistics:")
print(df_aoi.groupby('correct')['aoi_50'].describe())

## 6. Cut-and-Paste Validation Demo

In [ ]:
# Note: This requires cornea annotations
# For demo purposes, we'll create a simple example

from validation.cut_and_paste import CorneaAnnotation

# Create sample annotation (you would load these from file)
sample_annotation = CorneaAnnotation(
    image_path=sample_image.image_path,
    center_x=320,  # Example values
    center_y=240,
    major_axis=200,
    minor_axis=180,
    angle=0
)

# Initialize cut-and-paste validator
cut_paste_validator = CutAndPasteValidator(model)

# Load image
img = cv2.imread(sample_image.image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Extract cornea region
cornea, mask = cut_paste_validator.extract_cornea(img_rgb, sample_annotation)

# Visualize extraction
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(img_rgb)
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(mask, cmap='gray')
axes[1].set_title('Cornea Mask')
axes[1].axis('off')

axes[2].imshow(cornea)
axes[2].set_title('Extracted Cornea')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("Cornea extraction complete!")
print(f"Cornea region shape: {cornea.shape}")

## 7. Interactive Parameter Exploration

In [ ]:
# Interactive widget for exploring different layers
from ipywidgets import interact, IntSlider, Dropdown

def explore_gradcam(layer_idx=0, alpha=0.5):
    """Interactive function to explore GradCAM results"""
    layer = target_layers[layer_idx]
    
    # Get CAM for selected layer
    cam, metadata = cams[layer]
    
    # Create visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Original image
    ax1.imshow(img)
    ax1.set_title('Original Image')
    ax1.axis('off')
    
    # CAM overlay
    overlay = gradcam.visualize_cam(img, cam, alpha=alpha)
    ax2.imshow(overlay)
    ax2.set_title(f'Layer {layer} - AOI_50: {metadata["aoi_50"]:.3f}')
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()

# Create interactive widget
interact(explore_gradcam,
         layer_idx=IntSlider(min=0, max=len(target_layers)-1, step=1, value=0,
                            description='Layer:'),
         alpha=FloatSlider(min=0.0, max=1.0, step=0.1, value=0.5,
                          description='Alpha:'))

## 8. Export Results

In [ ]:
# Save analysis results
output_dir = Path("demo_results")
output_dir.mkdir(exist_ok=True)

# Save AOI results
df_aoi.to_csv(output_dir / "aoi_analysis.csv", index=False)
print(f"AOI results saved to {output_dir / 'aoi_analysis.csv'}")

# Save sample visualization
fig, ax = plt.subplots(figsize=(10, 8))
overlay = gradcam.visualize_cam(img, cams['23'][0], alpha=0.5)
ax.imshow(overlay)
ax.set_title(f'{sample_image.class_name} - Layer 23 GradCAM++')
ax.axis('off')
plt.savefig(output_dir / "sample_gradcam.png", dpi=300, bbox_inches='tight')
plt.close()

print(f"Sample visualization saved to {output_dir / 'sample_gradcam.png'}")

# Summary report
summary = {
    "n_images_analyzed": len(df_aoi),
    "accuracy": (df_aoi['correct'].sum() / len(df_aoi)),
    "mean_aoi_correct": df_aoi[df_aoi['correct']]['aoi_50'].mean(),
    "mean_aoi_incorrect": df_aoi[~df_aoi['correct']]['aoi_50'].mean(),
    "aoi_by_class": df_aoi.groupby('true_class')['aoi_50'].mean().to_dict()
}

print("\nAnalysis Summary:")
for key, value in summary.items():
    print(f"  {key}: {value}")

# Save to Google Drive if in Colab
if IN_COLAB:
    drive_path = "/content/drive/MyDrive/YOLOv5_GradCAM_Results"
    !mkdir -p {drive_path}
    !cp -r demo_results/* {drive_path}/
    print(f"\nResults also saved to Google Drive: {drive_path}")

In [ ]:
## 9. Conclusion

This demo notebook has shown how to use YOLOv5 GradCAM++ for anterior segment disease classification:

1. **Setup on Google Colab**: Easy installation and configuration
2. **Model Loading**: Load YOLOv5 model weights
3. **GradCAM++ Visualization**: Generate attention maps for different layers
4. **AOI_50 Analysis**: Calculate and compare attention metrics
5. **Cut-and-Paste Demo**: Extract corneal regions for validation
6. **Export Results**: Save results to Google Drive

### Next Steps

- Upload your own model weights and test images
- Run the full analysis pipeline using `main.py`
- Perform statistical tests on larger datasets
- Compare with expert annotations

### Resources

- [Paper]: YOLOv5 Attention Analysis for Anterior Eye Disease Classification
- [GitHub Repository]: https://github.com/kitaguchi77/yolov5-gradcam-corneai
- [YOLOv5 Documentation]: https://github.com/ultralytics/yolov5

For questions or issues, please refer to the repository's issue tracker.

## 9. Conclusion

This demo notebook has shown:

1. How to load and use the YOLOv5 model for anterior segment disease classification
2. How to generate and visualize Grad-CAM++ attention maps
3. How to calculate AOI_50 metrics
4. Basic cut-and-paste validation concepts
5. How to analyze results and generate reports

For full analysis including statistical tests and complete cut-and-paste validation, use the main.py script with appropriate data and annotations.